In [4]:
from duckduckgo_search import DDGS
from fastapi.responses import JSONResponse
import pandas as pd
import numpy as np
import time


df = pd.read_csv('/home/kkm/work/flyai/K-CAT/kkm/preprocessing/datasets/PJZ.csv', encoding="latin1")

# test chat
query = "Hello there!"
results = DDGS().chat(query, model='gpt-4o-mini')
print(results)

/tmp/ipykernel_1199/1545164459.py:8: DtypeWarning: Columns (7,11,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/home/kkm/work/flyai/K-CAT/kkm/preprocessing/datasets/PJZ.csv', encoding="latin1")


Hello! How can I assist you today?


In [151]:
# test chat
query = "ㅎㅇ"
results = DDGS().chat(query, model='gpt-4o-mini')
print(results)

안녕하세요! 어떻게 도와드릴까요?


In [ ]:
df = df.drop(df.columns[6:], axis=1)
df = df[df['label'] == 1]

,source,label,message_idx,author,time,text
0,PJ chats,1,0,decoy,14:40,Hey Its Mads
1,PJ chats,1,0,Billy Joe,14:40,Hey babes
2,PJ chats,1,0,Billy Joe,14:41,Almost done with work. So glad
3,PJ chats,1,0,decoy,14:41,Hey:) that's good right?
4,PJ chats,1,0,Billy Joe,14:42,Yea
...,...,...,...,...,...,...
20565,PJ chats,1,1103,decoy,18:20,it's cooling down I don't wanna stand outside ...
20566,PJ chats,1,1103,tory beltz,18:20,I understand
20567,PJ chats,1,1103,tory beltz,18:25,I'm by the water tower
20568,PJ chats,1,1103,decoy,18:28,ok on my way


In [91]:
df

,source,label,message_idx,author,time,text
0,PJ chats,1,0,decoy,14:40,Hey Its Mads
1,PJ chats,1,0,Billy Joe,14:40,Hey babes
2,PJ chats,1,0,Billy Joe,14:41,Almost done with work. So glad
3,PJ chats,1,0,decoy,14:41,Hey:) that's good right?
4,PJ chats,1,0,Billy Joe,14:42,Yea
...,...,...,...,...,...,...
20565,PJ chats,1,1103,decoy,18:20,it's cooling down I don't wanna stand outside ...
20566,PJ chats,1,1103,tory beltz,18:20,I understand
20567,PJ chats,1,1103,tory beltz,18:25,I'm by the water tower
20568,PJ chats,1,1103,decoy,18:28,ok on my way


In [133]:
df_save = pd.read_csv('/home/kkm/work/flyai/K-CAT/kkm/preprocessing/complete1.csv')
print(df['translated text'].iloc[800])
print(df['author'].iloc[800])
print(df['translated text'].iloc[0])
df['translated text'].iloc[:800] = df_save['translated text'].iloc[:800]

heyyy  
decoy
heyyy  


/tmp/ipykernel_1309/216244644.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['translated text'].iloc[:800] = df_save['translated text'].iloc[:800]


In [136]:
df['translated text'].iloc[2]

'Billy Joe: 거의 일 끝나가. 정말 기뻐  '

In [ ]:
# df_chat = pd.DataFrame()
max_message_idx = max(df['message_idx'])
num_of_chunks = 4
total_idx = 800
for i in range(15, max_message_idx):
    print('**********************************')
    print(f'message_idx: {i}')
    df_chat = df[df['message_idx'] == i].copy()
    df_chat.reset_index(drop=True, inplace=True)
    # message_idx마다 GPT와의 대화방 사이트 하나씩 파자
    ddgs_session = DDGS()
    total_chat_length = df_chat.shape[0]
    offset = total_chat_length // num_of_chunks # 최대토큰수가 넘어갈 수도 있으니까 총 길이의 1/num_of_chunks씩 넣기
    print(f'total_chat_length: {total_chat_length}, offset: {offset}')
    if total_chat_length % num_of_chunks == 0:
        max_chunk_iter = num_of_chunks
    else: 
        max_chunk_iter = num_of_chunks + 1
    print(f'max_chunk_iter: {max_chunk_iter}')
    for j in range(max_chunk_iter):
        chat_chunk = ""
        query_prompt = ""
        if j == 0:
            query_prompt = "\
                다음 대화 상황들을 자연스럽게 번역해줘. 대화 상황은 ㅁ문자로 구분된 문장들로 구성되어 있고,\
                     직전 문장과 의미가 비슷하거나 잘 못알아 듣더라도 최선을 다해서 ㅁ으로 구분된 모든 문장들을 반드시 빠짐없이 번역해야 해.\
                         대화하는 사람 이름도 넣고, 선정적인 단어는 덜 선정적이도록 변형해서 번역해줘.\
                        번역한 문장에 ㅁ는 붙이지마. 대답하지말고 바로 번역해줘."
        else:
            query_prompt = "이전에 번역했던 문장들의 문맥을 고려해서 다음 대화상황도 자연스럽게 번역해줘. ㅁ문자로 구분된 문장들로 구성되어 있고, \
                직전 문장과 의미가 비슷하거나 잘 못알아 듣더라도 최선을 다해서 ㅁ으로 구분된 모든 문장들을 반드시 빠짐없이 번역해야 해. \
                    대화하는 사람 이름도 넣고, 선정적인 단어는 덜 선정적이도록 변형해서 번역해줘.\
                    번역한 문장에 ㅁ는 붙이지마. 대답하지 말고 바로 번역해줘."
                    
        if j < max_chunk_iter - 1:
            max_iter = offset
        else: # 4개의 청크 중 마지막 청크는 남은거 모두 넣기
            max_iter = total_chat_length - j*offset
        print(f'max_iter: {max_iter}')
        
        for k in range(max_iter):
            print(f'current idx: {j*offset + k}, df_chat shape:{df_chat.shape}')
            chat_chunk += df_chat.loc[j*offset + k, 'author'] + ":" + df_chat.loc[j*offset + k, 'text'] + 'ㅁ'
        
        translated_text_chunk = ddgs_session.chat(query_prompt + chat_chunk, model='gpt-4o-mini')
        time.sleep(4)
        print(translated_text_chunk)
        splitted_chunks = translated_text_chunk.split('\n')
        print()
        print(f'splitted_chunks length: {len(splitted_chunks)}')
        print(splitted_chunks)
        print(f'idx: {total_idx + j*offset}')
        print('**********************************')
        for k in range(max_iter):
            df.loc[total_idx + j*offset + k, 'translated text'] = splitted_chunks[k]
    
    total_idx += total_chat_length # df에 저장할 위치 찾기위한 인덱스 total_idx
            
        # i=0 -> 120개, i=1 -> 95개

**********************************
message_idx: 15
total_chat_length: 71, offset: 17
max_chunk_iter: 5
max_iter: 17
current idx: 0, df_chat shape:(71, 7)
current idx: 1, df_chat shape:(71, 7)
current idx: 2, df_chat shape:(71, 7)
current idx: 3, df_chat shape:(71, 7)
current idx: 4, df_chat shape:(71, 7)
current idx: 5, df_chat shape:(71, 7)
current idx: 6, df_chat shape:(71, 7)
current idx: 7, df_chat shape:(71, 7)
current idx: 8, df_chat shape:(71, 7)
current idx: 9, df_chat shape:(71, 7)
current idx: 10, df_chat shape:(71, 7)
current idx: 11, df_chat shape:(71, 7)
current idx: 12, df_chat shape:(71, 7)
current idx: 13, df_chat shape:(71, 7)
current idx: 14, df_chat shape:(71, 7)
current idx: 15, df_chat shape:(71, 7)
current idx: 16, df_chat shape:(71, 7)
decoy: 헤이  
Carlos Cabrera: 안녕  
decoy: 뭐 하고 있어?  
Carlos Cabrera: 침대에 있어, 너는?  
decoy: TV 보고 있어  
Carlos Cabrera: 나도  
Carlos Cabrera: 너무 지루해  
decoy: 나도 ㅋㅋ  
Carlos Cabrera: 너와 함께 있고 싶어  
decoy: 나도 네가 여기 있으면 좋겠어 :) :P  
Carlos Ca

KeyboardInterrupt: 

In [157]:
# df_chat = pd.DataFrame()
max_message_idx = max(df['message_idx'])
num_of_chunks = 4
total_idx = 800
for i in range(15, max_message_idx):
    print('**********************************')
    print(f'message_idx: {i}')
    df_chat = df[df['message_idx'] == i].copy()
    df_chat.reset_index(drop=True, inplace=True)
    # message_idx마다 GPT와의 대화방 사이트 하나씩 파자
    ddgs_session = DDGS()
    total_chat_length = df_chat.shape[0]
    offset = total_chat_length // num_of_chunks # 최대토큰수가 넘어갈 수도 있으니까 총 길이의 1/num_of_chunks씩 넣기
    print(f'total_chat_length: {total_chat_length}, offset: {offset}')
    if total_chat_length % num_of_chunks == 0:
        max_chunk_iter = num_of_chunks
    else: 
        max_chunk_iter = num_of_chunks + 1
    print(f'max_chunk_iter: {max_chunk_iter}')
    for j in range(max_chunk_iter):
        chat_chunk = ""
        query_prompt = ""
        if j == 0:
            query_prompt = "\
                다음 대화 상황들을 자연스럽게 번역해줘. 대화 상황은 ㅁ문자로 구분된 문장들로 구성되어 있고,\
                     직전 문장과 의미가 비슷하거나 조금 이상한 문장이라도 최선을 다해서 ㅁ으로 구분된 모든 문장들을 반드시 빠짐없이 번역해야 해.\
                         대화하는 사람 이름도 넣고, 선정적인 단어는 덜 선정적이도록 번역해줘.\
                        번역한 문장에 ㅁ는 붙이지마. 대답하지말고 바로 번역해줘."
        else:
            query_prompt = "이전에 번역했던 문장들의 문맥을 고려해서 다음 대화상황도 자연스럽게 번역해줘. ㅁ문자로 구분된 문장들로 구성되어 있고, \
                직전 문장과 의미가 비슷하거나 조금 이상한 문장이라도 최선을 다해서 ㅁ으로 구분된 모든 문장들을 반드시 빠짐없이 번역해야 해. \
                    대화하는 사람 이름도 넣고, 선정적인 단어는 덜 선정적이도록 번역해줘.\
                    번역한 문장에 ㅁ는 붙이지마. 대답하지 말고 바로 번역해줘."
                    
        if j < max_chunk_iter - 1:
            max_iter = offset
        else: # 4개의 청크 중 마지막 청크는 남은거 모두 넣기
            max_iter = total_chat_length - j*offset
        print(f'max_iter: {max_iter}')
        
        for k in range(max_iter):
            print(f'current idx: {j*offset + k}, df_chat shape:{df_chat.shape}')
            chat_chunk += df_chat.loc[j*offset + k, 'author'] + ":" + df_chat.loc[j*offset + k, 'text'] + 'ㅁ'
        
        translated_text_chunk = ddgs_session.chat(query_prompt + chat_chunk, model='gpt-4o-mini')
        time.sleep(4)
        print(translated_text_chunk)
        splitted_chunks = translated_text_chunk.split('\n')
        print()
        print(f'splitted_chunks length: {len(splitted_chunks)}')
        print(splitted_chunks)
        print(f'last idx for saving message {i}: {total_idx + j*offset + len(splitted_chunks)}')
        print('**********************************')
        for k in range(max_iter):
            if k < len(splitted_chunks):
                df.loc[total_idx + j*offset + k, 'translated text'] = splitted_chunks[k]
            else:
                df.loc[total_idx + j*offset + k, 'translated text'] =  ""
    
    total_idx += total_chat_length # df에 저장할 위치 찾기위한 인덱스 total_idx

**********************************
message_idx: 15
total_chat_length: 71, offset: 17
max_chunk_iter: 5
max_iter: 17
current idx: 0, df_chat shape:(71, 7)
current idx: 1, df_chat shape:(71, 7)
current idx: 2, df_chat shape:(71, 7)
current idx: 3, df_chat shape:(71, 7)
current idx: 4, df_chat shape:(71, 7)
current idx: 5, df_chat shape:(71, 7)
current idx: 6, df_chat shape:(71, 7)
current idx: 7, df_chat shape:(71, 7)
current idx: 8, df_chat shape:(71, 7)
current idx: 9, df_chat shape:(71, 7)
current idx: 10, df_chat shape:(71, 7)
current idx: 11, df_chat shape:(71, 7)
current idx: 12, df_chat shape:(71, 7)
current idx: 13, df_chat shape:(71, 7)
current idx: 14, df_chat shape:(71, 7)
current idx: 15, df_chat shape:(71, 7)
current idx: 16, df_chat shape:(71, 7)
decoy: 안녕  
Carlos Cabrera: 안녕하세요  
decoy: 뭐 하고 있어?  
Carlos Cabrera: 침대에 있어, 너는?  
decoy: TV 보고 있어  
Carlos Cabrera: 나도  
Carlos Cabrera: 너무 지루해  
decoy: 나도 ㅋㅋ  
Carlos Cabrera: 너와 함께 있고 싶어  
decoy: 나도 네가 여기 있었으면 좋겠어 :) :P  
Carlo

KeyboardInterrupt: 